<a href="https://colab.research.google.com/github/yogisaireddy14/Intellifailabs_assignment/blob/main/Intellifailabs_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/KwaiVGI/LivePortrait.git
%cd LivePortrait

fatal: destination path 'LivePortrait' already exists and is not an empty directory.
/content/LivePortrait


In [2]:
!pip install -q --upgrade pip
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.2 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB

In [2]:
!pip install transformers==4.41.0
!pip install numpy==1.24.4 onnxruntime-gpu==1.17.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 125.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.0
    Uninstalling transformers-4.38.0:
      Successfully uninstalled transformers-4.38.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 140.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 63.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: onnxruntime-gpu
    Found existing installation: onnxruntime-gpu 1.18.0
    Uninstalling onnxru

In [2]:
import torch
print("PyTorch CUDA available? ", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Current device:", torch.cuda.get_device_name(0))
    print("CUDA capability:", torch.version.cuda)
else:
    print("No GPU detected.")

PyTorch CUDA available?  True
Current device: Tesla T4
CUDA capability: 12.1


In [3]:
!huggingface-cli download KwaiVGI/LivePortrait \
  --local-dir pretrained_weights \
  --exclude "*.git*" "README.md" "docs"

Fetching 20 files:   0% 0/20 [00:00<?, ?it/s]Downloading 'liveportrait/base_models/spade_generator.pth' to 'pretrained_weights/.cache/huggingface/download/liveportrait/base_models/7eKjApaVmkkjxwgjxli7XtNsnxU=.4780afc7909a9f84e24c01d73b31a555ef651521a1fe3b2429bd04534d992aee.incomplete'

spade_generator.pth:   0% 0.00/222M [00:00<?, ?B/s]Downloading 'liveportrait/base_models/appearance_feature_extractor.pth' to 'pretrained_weights/.cache/huggingface/download/liveportrait/base_models/w2JquOgGo_-zOZp03SpATaJOQmc=.5279bb8654293dbdf327030b397f107237dd9212fb11dd75b83dfb635211ceb5.incomplete'


warping_module.pth:   0% 0.00/182M [00:00<?, ?B/s]Downloading 'docs/inference.gif' to 'pretrained_weights/.cache/huggingface/download/docs/ke8qy6KU2hXc8UMbe9pzGl7iH4Q=.e1316eca5556ba5a8da7c53bcadbc1df26aa822bbde68fbad94813139803d0c6.incomplete'



motion_extractor.pth:   0% 0.00/113M [00:00<?, ?B/s]



showcase2.gif:   0% 0.00/2.88M [00:00<?, ?B/s]Downloading 'insightface/models/buffalo_l/2d106det.onnx'

In [4]:
import torch
_ = torch.zeros((1,1), device="cuda")
torch.cuda.synchronize()
torch.cuda.empty_cache()

In [5]:
!nvidia-smi --query-gpu=memory.used --format=csv,nounits,noheader

120


In [6]:
import time
import subprocess
import sys
SRC_IMG = "/content/LivePortrait/assets/examples/source/s9.jpg"
DRIVE_VID = "/content/LivePortrait/assets/examples/driving/d0.mp4"
OUT_DIR = "animations"

start_time = time.time()
try:
    result = subprocess.run(
        [
            sys.executable, "/content/LivePortrait/inference.py",
            "-s", SRC_IMG,
            "-d", DRIVE_VID,
            "-o", OUT_DIR
        ],
        check=True,
        capture_output=True,
        text=True
    )

except subprocess.CalledProcessError as e:
    print(" inference.py exited with a non-zero status.")
    print("------ stdout ------")
    print(e.stdout)
    print("------ stderr ------")
    print(e.stderr)
    raise

end_time = time.time()

elapsed_ms = (end_time - start_time) * 1000
print(f"=== Elapsed inference time (FP32): {elapsed_ms:.0f} ms total ===")


=== Elapsed inference time (FP32): 47294 ms total ===


In [7]:
import torch
from src.modules.spade_generator import SPADEDecoder  # Adjust this path if needed

generator = SPADEDecoder()
generator.eval().cuda()

ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/base_models/spade_generator.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")

generator.load_state_dict(state_dict, strict=False)

dummy_input = torch.randn(512, 256, 3, 3).cuda()
with torch.cuda.amp.autocast(), torch.no_grad():
    output = generator(dummy_input)

print(" Inference complete. Output shape:", output.shape)

traced = torch.jit.trace(generator, dummy_input)
torchscript_path = "/content/spade_generator_torchscript.pt"
traced.save(torchscript_path)
print(f" TorchScript model saved at: {torchscript_path}")

 Inference complete. Output shape: torch.Size([512, 3, 12, 12])
 TorchScript model saved at: /content/spade_generator_torchscript.pt


In [9]:
for _ in range(10):
    _ = generator(dummy_input)

# Time it
start = time.time()
for _ in range(100):
    _ = generator(dummy_input)
end = time.time()

print(f"Original Model Avg Inference Time: {(end - start)/100:.6f} sec")
optimized_model = torch.jit.load("/content/spade_generator_torchscript.pt").eval().cuda()

# Warm-up
for _ in range(10):
    _ = optimized_model(dummy_input)

# Time it
start = time.time()
for _ in range(100):
    _ = optimized_model(dummy_input)
end = time.time()

print(f"TorchScript Model Avg Inference Time: {(end - start)/100:.6f} sec")

Original Model Avg Inference Time: 0.364458 sec
TorchScript Model Avg Inference Time: 0.331911 sec


In [11]:
import torch
from src.modules.motion_extractor import MotionExtractor

generator = MotionExtractor(num_kp=21)
generator.eval().cuda()

ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/base_models/motion_extractor.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")
generator.load_state_dict(state_dict, strict=True)

dummy_input = torch.randn(1,3,256,256).cuda()

with torch.cuda.amp.autocast(), torch.no_grad():
    output = dict(generator(dummy_input))

print(" Inference complete. Output shape:", output)


traced = torch.jit.trace(generator, dummy_input, strict=False)
torchscript_path = "/content/motion_extractor_torchscript.pt"
traced.save(torchscript_path)
print(f" TorchScript model saved at: {torchscript_path}")


 Inference complete. Output shape: {'pitch': tensor([[ 1.3418,  1.3535,  1.3408,  1.2930,  1.2197,  1.1113,  0.9795,  0.8403,
          0.6836,  0.5117,  0.3328,  0.1509, -0.0259, -0.1992, -0.3647, -0.5127,
         -0.6436, -0.7510, -0.8281, -0.8672, -0.8652, -0.7993, -0.6763, -0.4839,
         -0.2725, -0.1049, -0.1070, -0.4238, -1.0283, -1.7207, -2.2930, -2.5840,
         -2.4121, -3.5781, -3.5215, -3.2383, -2.4141, -1.6260, -1.1973, -1.0576,
         -1.0459, -1.1006, -1.1885, -1.2588, -1.2939, -1.2881, -1.2412, -1.1631,
         -1.0635, -0.9541, -0.8379, -0.7188, -0.5996, -0.4822, -0.3679, -0.2576,
         -0.1501, -0.0508,  0.0456,  0.1295,  0.2047,  0.2712,  0.3335,  0.3904,
          0.4426,  0.4905]], device='cuda:0', dtype=torch.float16), 'yaw': tensor([[-3.9883, -2.9707, -1.9854, -1.0771, -0.2788,  0.4021,  0.9668,  1.4219,
          1.7754,  2.0332,  2.2031,  2.2930,  2.3086,  2.2578,  2.1602,  2.0352,
          1.8838,  1.7148,  1.5293,  1.3281,  1.1123,  0.8779,  0.6245

In [12]:
for _ in range(10):
    _ = generator(dummy_input)

start = time.time()
for _ in range(100):
    _ = generator(dummy_input)
end = time.time()

print(f"Original Model Avg Inference Time: {(end - start)/100:.6f} sec")
optimized_model = torch.jit.load("/content/motion_extractor_torchscript.pt").eval().cuda()


for _ in range(10):
    _ = optimized_model(dummy_input)


start = time.time()
for _ in range(100):
    _ = optimized_model(dummy_input)
end = time.time()

print(f"TorchScript Model Avg Inference Time: {(end - start)/100:.6f} sec")

Original Model Avg Inference Time: 0.011086 sec
TorchScript Model Avg Inference Time: 0.007991 sec


In [13]:
import torch
from src.modules.appearance_feature_extractor import AppearanceFeatureExtractor


model = AppearanceFeatureExtractor(
    image_channel=3,
    block_expansion=64,
    num_down_blocks=2,
    max_features=512,
    reshape_channel=32,
    reshape_depth=16,
    num_resblocks=6
)
model.eval().cuda()


ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/base_models/appearance_feature_extractor.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")
model.load_state_dict(state_dict, strict=True)


dummy_input = torch.randn(1, 3, 256, 256).cuda()


traced = torch.jit.trace(model, dummy_input, strict=False)


torchscript_path = "/content/appearance_feature_extractor.pt"
traced.save(torchscript_path)

print(f" TorchScript model saved at: {torchscript_path}")


 TorchScript model saved at: /content/appearance_feature_extractor.pt


In [14]:
for _ in range(10):
    _ = generator(dummy_input)


start = time.time()
for _ in range(100):
    _ = generator(dummy_input)
end = time.time()

print(f"Original Model Avg Inference Time: {(end - start)/100:.6f} sec")
optimized_model = torch.jit.load("/content/appearance_feature_extractor.pt").eval().cuda()


for _ in range(10):
    _ = optimized_model(dummy_input)


start = time.time()
for _ in range(100):
    _ = optimized_model(dummy_input)
end = time.time()

print(f"TorchScript Model Avg Inference Time: {(end - start)/100:.6f} sec")

Original Model Avg Inference Time: 0.011196 sec
TorchScript Model Avg Inference Time: 0.024077 sec


In [15]:
import torch
from src.modules.stitching_retargeting_network import StitchingRetargetingNetwork  # Adjust this path if needed


generator = StitchingRetargetingNetwork( input_size=256,
    hidden_sizes=[512, 512, 256],
    output_size=3)
generator.eval().cuda()


ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/retargeting_models/stitching_retargeting_module.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")
for name, param in generator.named_parameters():
    print(name, param.shape)

generator.load_state_dict(state_dict, strict=False)


dummy_input = torch.randn(1, 256).cuda()
with torch.cuda.amp.autocast(), torch.no_grad():
    output = generator(dummy_input)

print(" Inference complete. Output shape:", output.shape)


traced = torch.jit.trace(generator, dummy_input)
torchscript_path = "/content/stitching_retargeting_torchscript.pt"
traced.save(torchscript_path)
print(f" TorchScript model saved at: {torchscript_path}")

mlp.0.weight torch.Size([512, 256])
mlp.0.bias torch.Size([512])
mlp.2.weight torch.Size([512, 512])
mlp.2.bias torch.Size([512])
mlp.4.weight torch.Size([256, 512])
mlp.4.bias torch.Size([256])
mlp.6.weight torch.Size([3, 256])
mlp.6.bias torch.Size([3])
 Inference complete. Output shape: torch.Size([1, 3])
 TorchScript model saved at: /content/stitching_retargeting_torchscript.pt


In [16]:
import torch
import time
from src.modules.stitching_retargeting_network import StitchingRetargetingNetwork  # Adjust this path if needed


generator = StitchingRetargetingNetwork( input_size=256,
    hidden_sizes=[512, 512, 256],
    output_size=3)
ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/retargeting_models/stitching_retargeting_module.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")
generator.load_state_dict(state_dict, strict=False)
generator.eval().cuda()
dummy_input = torch.randn(1, 256).cuda()
for _ in range(10):
    _ = generator(dummy_input)


start = time.time()
for _ in range(100):
    _ = generator(dummy_input)
end = time.time()

print(f"Original Model Avg Inference Time: {(end - start)/100:.6f} sec")
optimized_model = torch.jit.load("/content/stitching_retargeting_torchscript.pt").eval().cuda()


for _ in range(10):
    _ = optimized_model(dummy_input)

start = time.time()
for _ in range(100):
    _ = optimized_model(dummy_input)
end = time.time()

print(f"TorchScript Model Avg Inference Time: {(end - start)/100:.6f} sec")

Original Model Avg Inference Time: 0.000181 sec
TorchScript Model Avg Inference Time: 0.000131 sec


In [17]:
import time
import os
import torch
from src.config.inference_config import InferenceConfig
from src.config.crop_config import CropConfig
from src.live_portrait_pipeline import LivePortraitPipeline
from IPython.display import Video, display
from src.config.argument_config import ArgumentConfig

SRC_IMG = "/content/LivePortrait/assets/examples/source/s9.jpg"
DRIVE_VID = "/content/LivePortrait/assets/examples/driving/d0.mp4"
OUT_DIR = "animations"
os.makedirs(OUT_DIR, exist_ok=True)


parsed_args = ArgumentConfig(source=SRC_IMG, driving=DRIVE_VID, output_dir=OUT_DIR)

pipeline = LivePortraitPipeline(inference_cfg=InferenceConfig(), crop_cfg=CropConfig())

start_time = time.time()

with torch.cuda.amp.autocast(), torch.no_grad():
    animation = pipeline.execute(parsed_args)


end_time = time.time()

elapsed_ms = (end_time - start_time) * 1000
print(f"=== Elapsed inference time (AMP only): {elapsed_ms:.0f} ms total ===")


[05:19:37] Load appearance_feature_extractor from                                       ]8;id=61433;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=755043;file:///content/LivePortrait/src/live_portrait_wrapper.py#46\46]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/base_models/appearance                            
           _feature_extractor.pth done.                                                                            

           Load motion_extractor from                                                   ]8;id=455017;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=522845;file:///content/LivePortrait/src/live_portrait_wrapper.py#49\49]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/base_models/motion_ext                            
           ractor.pth done.                                                                                        

[05:19:38] Load warping_module from                                                     ]8;id=351275;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=843950;file:///content/LivePortrait/src/live_portrait_wrapper.py#52\52]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/base_models/warping_mo                            
           dule.pth done.                                                                                          

[05:19:39] Load spade_generator from                                                    ]8;id=206954;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=186778;file:///content/LivePortrait/src/live_portrait_wrapper.py#55\55]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/base_models/spade_gene                            
           rator.pth done.                                                                                         

           Load stitching_retargeting_module from                                       ]8;id=245280;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=964697;file:///content/LivePortrait/src/live_portrait_wrapper.py#59\59]8;;\
           /content/LivePortrait/pretrained_weights/liveportrait/retargeting_models/sti                            
           tching_retargeting_module.pth done.                                                                     

...

Output()

[05:20:18] Animated template: /content/LivePortrait/assets/examples/driving/d0.pkl,   ]8;id=132926;file:///content/LivePortrait/src/live_portrait_pipeline.py\live_portrait_pipeline.py]8;;\:]8;id=334611;file:///content/LivePortrait/src/live_portrait_pipeline.py#503\503]8;;\
           you can specify `-d` argument with this template path next time to avoid                                
           cropping video, motion making and protecting privacy.                                                   

           Animated video: animations/s9--d0.mp4                                      ]8;id=539288;file:///content/LivePortrait/src/live_portrait_pipeline.py\live_portrait_pipeline.py]8;;\:]8;id=603762;file:///content/LivePortrait/src/live_portrait_pipeline.py#504\504]8;;\

           Animated video with concat: animations/s9--d0_concat.mp4                   ]8;id=824523;file:///content/LivePortrait/src/live_portrait_pipeline.py\live_portrait_pipeline.py]8;;\:]8;id=64463;file:///content/LivePortrait/src/live_portrait_pipeline.py#505\505]8;;\

=== Elapsed inference time (AMP only): 38124 ms total ===
